# ML and DL Based LLM-Human Generated Text Classification

#### Harun Harman- 2200765008
#### Mehmet Eren Soykök- 22007650013

This file is the dataset loading, preprocessing and creating the final version of training set file.

### Importing Libraries

In [13]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import string
from gensim.models import Word2Vec

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


#### Loading Data

In [2]:
df = pd.read_csv("D:/AIN423 PROJECT/Training_Essay_Data.csv")
df1 = pd.read_csv("D:/AIN423 PROJECT/train_drcat_01.csv")
df2 = pd.read_csv("D:/AIN423 PROJECT/train_drcat_02.csv")
df3 = pd.read_csv("D:/AIN423 PROJECT/train_drcat_03.csv")
df4 = pd.read_csv("D:/AIN423 PROJECT/train_drcat_04.csv")

In [3]:
df.rename(columns = {'generated':'label'}, inplace = True)
df

,text,label
0,Car-free cities have become a subject of incre...,1
1,"Car Free Cities Car-free cities, a concept ga...",1
2,A Sustainable Urban Future Car-free cities ...,1
3,Pioneering Sustainable Urban Living In an e...,1
4,The Path to Sustainable Urban Living In an ...,1
...,...,...
29140,There has been a fuss about the Elector Colleg...,0
29141,Limiting car usage has many advantages. Such a...,0
29142,There's a new trend that has been developing f...,0
29143,As we all know cars are a big part of our soci...,0


In [4]:
df1 = df1.drop(columns = ["source", "fold"])
df1

,text,label
0,There are alot reasons to keep our the despise...,0
1,Driving smart cars that drive by themself has ...,0
2,"Dear Principal,\n\nI believe that students at ...",0
3,"Dear Principal,\n\nCommunity service should no...",0
4,My argument for the development of the driverl...,0
...,...,...
33254,Are driverless cars really necessary? Most of ...,0
33255,"""Oh man I didn't make the soccer team!"", yelle...",0
33256,I believe that using this technology could be ...,0
33257,Texting & Driving\n\nUsing your phone while dr...,0


In [5]:
df2 = df2.drop(columns = ["source", "fold", "essay_id", "prompt"])
df2

,text,label
0,"Okay, here is my essay:\n\nWorking or doing th...",1
1,"Dear Principal,\r\n\r\nI just want to let you ...",0
2,I believe they should change the voting to pop...,0
3,One example of an inspiring individual who has...,1
4,"Dear Mr. Principal,\n\nI believe that students...",0
...,...,...
39780,"""Oh man I didn't make the soccer team!"", yelle...",0
39781,I believe that using this technology could be ...,0
39782,The Face on Mars is a fascinating phenomenon t...,1
39783,Texting & Driving\n\nUsing your phone while dr...,0


In [6]:
df3 = df3.drop(columns = ["source", "fold", "essay_id", "prompt"])
df3

,text,label
0,People can learn without making mistakes. Peop...,0
1,PHONES AND DRIVING\n\nIn this world in which w...,0
2,"Okay, here's my essay:\n\nMaking Your Own Deci...",1
3,Dear : Principal\n\nI believe that allowing s...,1
4,Well for one if you seek more then one person ...,0
...,...,...
42201,"""Oh man I didn't make the soccer team!"", yelle...",0
42202,I believe that using this technology could be ...,0
42203,The Face on Mars is a fascinating phenomenon t...,1
42204,Texting & Driving\n\nUsing your phone while dr...,0


In [7]:
df4 = df4.drop(columns = ["source", "fold", "essay_id", "prompt"])
df4

,text,label
0,"In recent years, technology has had a profoun...",1
1,Should students participate in an extracurricu...,0
2,The electoral college is a symbol of mockery a...,0
3,This is why I think the principle should allow...,0
4,I strongly believe that meditation and mindful...,1
...,...,...
44201,"""Oh man I didn't make the soccer team!"", yelle...",0
44202,I believe that using this technology could be ...,0
44203,The Face on Mars is a fascinating phenomenon t...,1
44204,Texting & Driving\n\nUsing your phone while dr...,0


In [8]:
df = pd.concat([df,df1, df2, df3, df4], ignore_index=True)
df

,text,label
0,Car-free cities have become a subject of incre...,1
1,"Car Free Cities Car-free cities, a concept ga...",1
2,A Sustainable Urban Future Car-free cities ...,1
3,Pioneering Sustainable Urban Living In an e...,1
4,The Path to Sustainable Urban Living In an ...,1
...,...,...
188596,"""Oh man I didn't make the soccer team!"", yelle...",0
188597,I believe that using this technology could be ...,0
188598,The Face on Mars is a fascinating phenomenon t...,1
188599,Texting & Driving\n\nUsing your phone while dr...,0


#### NaN Check

In [9]:
nan_counts = df.isna().sum()
nan_counts

text     0
label    0
dtype: int64

#### Turning every word into lowercase

In [10]:
def lowercase_words(cell):
    return ' '.join(word.lower() for word in cell.split())

# Applying the function to the DataFrame
df['text'] = df['text'].apply(lowercase_words)
df

,text,label
0,car-free cities have become a subject of incre...,1
1,"car free cities car-free cities, a concept gai...",1
2,a sustainable urban future car-free cities are...,1
3,pioneering sustainable urban living in an era ...,1
4,the path to sustainable urban living in an age...,1
...,...,...
188596,"""oh man i didn't make the soccer team!"", yelle...",0
188597,i believe that using this technology could be ...,0
188598,the face on mars is a fascinating phenomenon t...,1
188599,texting & driving using your phone while drivi...,0


#### Removing punctuation and special characters

In [11]:
def remove_punctuation(cell):
    return ''.join(char for char in cell if char not in string.punctuation)

# Applying the function to the DataFrame
df['text'] = df['text'].apply(remove_punctuation)
df

,text,label
0,carfree cities have become a subject of increa...,1
1,car free cities carfree cities a concept gaini...,1
2,a sustainable urban future carfree cities are ...,1
3,pioneering sustainable urban living in an era ...,1
4,the path to sustainable urban living in an age...,1
...,...,...
188596,oh man i didnt make the soccer team yelled gen...,0
188597,i believe that using this technology could be ...,0
188598,the face on mars is a fascinating phenomenon t...,1
188599,texting driving using your phone while drivin...,0


#### Tokenization

In [12]:
def tokenize(cell):
    return word_tokenize(cell)
df['Tokenized_text'] = df['text'].apply(tokenize)
df

,text,label,Tokenized_text
0,carfree cities have become a subject of increa...,1,"[carfree, cities, have, become, a, subject, of..."
1,car free cities carfree cities a concept gaini...,1,"[car, free, cities, carfree, cities, a, concep..."
2,a sustainable urban future carfree cities are ...,1,"[a, sustainable, urban, future, carfree, citie..."
3,pioneering sustainable urban living in an era ...,1,"[pioneering, sustainable, urban, living, in, a..."
4,the path to sustainable urban living in an age...,1,"[the, path, to, sustainable, urban, living, in..."
...,...,...,...
188596,oh man i didnt make the soccer team yelled gen...,0,"[oh, man, i, didnt, make, the, soccer, team, y..."
188597,i believe that using this technology could be ...,0,"[i, believe, that, using, this, technology, co..."
188598,the face on mars is a fascinating phenomenon t...,1,"[the, face, on, mars, is, a, fascinating, phen..."
188599,texting driving using your phone while drivin...,0,"[texting, driving, using, your, phone, while, ..."


In [14]:
df.to_csv('D:\\AIN423 PROJECT\\tokenized.csv')

#### Lemmatization

In [15]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default to noun if tag not found

tagged_tokens = nltk.pos_tag_sents(df['Tokenized_text'].tolist())
tagged_tokens = [tagged_token for sublist in tagged_tokens for tagged_token in sublist]  # flatten the list of lists

# Lemmatization with POS tagging
lemmatizer = WordNetLemmatizer()

def lemmatize_with_pos(token, tag):
    wn_tag = get_wordnet_pos(tag)
    if wn_tag is None:
        return token
    else:
        return lemmatizer.lemmatize(token, pos=wn_tag)

df['Lemmatized_tokens'] = df['Tokenized_text'].apply(lambda x: [lemmatize_with_pos(token, tag) for token, tag in nltk.pos_tag(x)])
df

,text,label,Tokenized_text,Lemmatized_tokens
0,carfree cities have become a subject of increa...,1,"[carfree, cities, have, become, a, subject, of...","[carfree, city, have, become, a, subject, of, ..."
1,car free cities carfree cities a concept gaini...,1,"[car, free, cities, carfree, cities, a, concep...","[car, free, city, carfree, city, a, concept, g..."
2,a sustainable urban future carfree cities are ...,1,"[a, sustainable, urban, future, carfree, citie...","[a, sustainable, urban, future, carfree, city,..."
3,pioneering sustainable urban living in an era ...,1,"[pioneering, sustainable, urban, living, in, a...","[pioneer, sustainable, urban, living, in, an, ..."
4,the path to sustainable urban living in an age...,1,"[the, path, to, sustainable, urban, living, in...","[the, path, to, sustainable, urban, living, in..."
...,...,...,...,...
188596,oh man i didnt make the soccer team yelled gen...,0,"[oh, man, i, didnt, make, the, soccer, team, y...","[oh, man, i, didnt, make, the, soccer, team, y..."
188597,i believe that using this technology could be ...,0,"[i, believe, that, using, this, technology, co...","[i, believe, that, use, this, technology, coul..."
188598,the face on mars is a fascinating phenomenon t...,1,"[the, face, on, mars, is, a, fascinating, phen...","[the, face, on, mar, be, a, fascinating, pheno..."
188599,texting driving using your phone while drivin...,0,"[texting, driving, using, your, phone, while, ...","[texting, drive, use, your, phone, while, driv..."


In [16]:
df.to_csv('D:\\AIN423 PROJECT\\data.csv')

#### Word2vec Encoding

In [16]:
# Train Word2Vec model (you can use pre-trained embeddings as well)
word2vec_model = Word2Vec(df['Lemmatized_tokens'], min_count=1)

# Get word vectors
word_vectors = word2vec_model.wv

# Convert documents to average word vectors
word_embeddings = []
for tokens in df['Lemmatized_tokens']:
    vectors = [word_vectors[word] for word in tokens if word in word_vectors]
    if vectors:
        word_embeddings.append(sum(vectors) / len(vectors))
    else:
        word_embeddings.append([0] * word2vec_model.vector_size)  # Placeholder for empty documents

# Convert list of vectors to numpy array
word_embeddings_array = np.array(word_embeddings)

In [17]:
word_embeddings_array.shape

(188601, 100)

In [18]:
embeddings_df = pd.DataFrame(word_embeddings_array)

# Concatenate with labels
final_df = pd.concat([embeddings_df, df['label']], axis=1)
final_df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,label
0,-0.271433,-0.382481,-0.317552,-0.021108,0.158111,0.887973,1.029558,0.240977,-0.557564,-0.207163,...,-0.532850,-0.324810,-0.180836,-0.748109,-0.011898,0.844318,0.209863,0.645236,-0.117491,1
1,-0.315431,-0.339699,-0.254841,0.008619,0.296162,0.900232,0.998118,0.170231,-0.487540,-0.217951,...,-0.469319,-0.419600,-0.270448,-0.662818,0.010100,0.852924,0.129529,0.621138,-0.107042,1
2,-0.343649,-0.281702,-0.231206,-0.052648,0.401156,0.945870,1.045683,0.093843,-0.495538,-0.228164,...,-0.409202,-0.295790,-0.062960,-0.589731,-0.045811,0.953600,0.109758,0.660381,-0.087883,1
3,-0.350994,-0.354819,-0.311409,0.025629,0.157560,0.993373,0.979554,0.165451,-0.444625,-0.249480,...,-0.338503,-0.334385,-0.178413,-0.663440,-0.125274,0.896376,0.167687,0.514862,-0.131908,1
4,-0.280636,-0.329962,-0.256767,0.053598,0.245001,0.927688,0.975017,0.192176,-0.439704,-0.278913,...,-0.374527,-0.310692,-0.085562,-0.691941,-0.116870,0.975996,0.187763,0.525309,-0.116779,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188596,0.313921,-0.017318,0.359710,0.017367,-0.271001,0.085610,-0.977056,-0.956168,0.917814,-0.628318,...,-1.105132,0.399504,1.049912,0.235927,-0.114889,-0.326360,0.605113,0.161475,-0.555144,0
188597,0.502092,0.080091,0.318333,0.351545,-0.455693,0.166257,-0.183564,-0.896472,1.015973,-0.524323,...,-0.228196,-0.341810,0.621761,0.985171,0.486032,0.180535,0.902042,0.152828,-1.025601,0
188598,0.009335,-0.509349,-0.023265,0.241166,-0.596270,0.856653,-0.383929,-0.456435,0.464538,-0.114828,...,0.218670,0.123300,-0.063834,0.453088,-0.414563,0.629535,0.330233,-0.232212,-0.997967,1
188599,-0.256354,-0.905941,0.438041,0.749837,-0.851218,-0.030804,-0.408960,-0.776891,1.045415,-0.563419,...,-1.158375,-0.027693,0.031016,0.407978,0.056792,-0.217032,0.888370,0.118055,0.404670,0


In [19]:
final_df.to_csv('D:\\AIN423 PROJECT\\final_dataset.csv')

**Disclaimer!** <font color='grey'>This notebook was prepared by <Harun Harman and Mehmet Eren Soykök> as a term project for the *AIN423 - Data Intensive Applications* class. The notebook is available for educational purposes only. There is no guarantee on the correctness of the content provided as it is a student work.

If you think there is any copyright violation, please let us [know](https://forms.gle/BNNRB2kR8ZHVEREq8). 
</font>